# ✅Step 6: Analysis of Ingredients and Cuisines

## 🎯0. Import libraries and load data

In [41]:
import pandas as pd          
import numpy as np 
from tqdm import tqdm
import sys
from plotnine import *
import altair as alt

# Import our own modules
sys.path.append("../scripts/")
import chadtools

In [4]:
df = pd.read_json("../data/merged_data_for_analysis.json", orient="records")
df.head(3)

,id,title,gpt_ingredients,cuisine,ingredient_comment,score,upvote_ratio,link_flair_text,author,created_utc,...,bbcgf_title,calories,salt,fat,sugars,saturates,carbs,protein,fibre,bbcgf_ratings
0,1ah8m5s,Thai Green Curry Chicken Satay,"[chicken satay, homemade green curry paste, ve...",thai,You could use any curry paste you like. Do you...,49,0.879883,Recipe,butchec,1706893611000,...,Thai green chicken curry,257,0.60,15.0,3.0,10.0,9.0,19.0,2.0,0.90
1,1afoma3,Sweet and Sour Tofu,"[tofu, ground black pepper, salt, cornstarch, ...",chinese,I love figuring out ways to make tofu deliciou...,59,0.890137,Recipe,parisrosaries,1706725321000,...,Sweet & sour tofu,530,1.20,17.0,18.0,2.0,75.0,15.0,8.0,0.82
2,1acagoz,Chocolate Fudgy Brownie,"[butter, dark chocolate, cocoa powder, white s...",american,Full Chocolate Fudgy Brownie recipe: https://w...,114,0.910156,Recipe,butchec,1706360858000,...,Fudgy brownies,1043,0.62,62.0,76.0,35.0,105.0,14.0,6.0,0.86


## 🎯1. Analysis of Ingredients

We first do some ingredient analysis for all the recipes. In particular, we are interested in finding how much of the *top 10 most common ingredients* are used in each cuisine.

In [37]:
df["cuisine"].value_counts().head(10)
frequency_table = df["gpt_ingredients"].value_counts()

all_ingredient_list = []
for i in df["gpt_ingredients"].tolist():
    all_ingredient_list.extend(i)

all_ingredient_series = pd.Series(all_ingredient_list)
top_10_ingredients = all_ingredient_series.value_counts().head(10)

In [53]:
ingredient_frequency_df = pd.DataFrame(top_10_ingredients).reset_index()
ingredient_frequency_df.columns = ["ingredient", "frequency"]
ingredient_frequency_df = ingredient_frequency_df.sort_values("frequency", ascending=False)

In [55]:
plot = (ggplot(ingredient_frequency_df, aes(x="ingredient", y="frequency")) +
       geom_bar(stat="identity", fill='#5c3da4') +
       coord_flip() +
       scale_x_discrete(limits=ingredient_frequency_df["ingredient"][::-1]) +
       theme(plot_title = element_text(weight='bold', color="black", size=10, ha="right")) +
       theme(aspect_ratio=9/16) +
       labs(title="Top 10 ingredients by frequency in both BBC GoodFood and Reddit recipes", 
            x="Ingredient", y="Number of recipes")
)
plot.save("../plots/top_10_ingredients_frequency.jpg", format="jpg", dpi=600)

c:\Users\Zicheng Liu\.conda\envs\ds105\lib\site-packages\plotnine\ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
c:\Users\Zicheng Liu\.conda\envs\ds105\lib\site-packages\plotnine\ggplot.py:588: PlotnineWarning: Filename: ../plots/top_10_ingredients_frequency.jpg
c:\Users\Zicheng Liu\.conda\envs\ds105\lib\site-packages\mizani\scale.py:210: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
c:\Users\Zicheng Liu\.conda\envs\ds105\lib\site-packages\mizani\scale.py:223: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


Hence, the top 10 ingredients found in recipes appearing on both BBC GoodFood and Reddit are:
1. Salt
2. Pepper
3. Garlic
4. Butter
5. Sugar
6. Olive oil
7. Water
8. Black Pepper
9. Eggs
10. Yellow Onion

## 🎯2. Analysis of Cuisines

We are also interested in the breakdown of cuisines in the recipes.

In [57]:
df["cuisine"].value_counts().head(10)

cuisine
italian     111
american     70
chinese      35
thai         32
french       25
japanese     21
indian       18
british      17
mexican      16
greek        12
Name: count, dtype: int64